In [16]:
import pandas as pd
import numpy as np
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F

Data Processing Prepare Training Data Set for DNN solution. 
For the data of 33 person, we chose 30 for training and 3 for validation. 
First Let's parse csvs.

In [2]:
def get_second_data(df):
    df['time'] = pd.to_datetime(df['time'])   # convert 'time' to datetime object
    df['time'] = df['time'].dt.floor("S")
    df['Second'] = df['time'].dt.second
    df['Minute'] = df['time'].dt.minute
    df_second = df.groupby(['Minute','Second']).mean()
    return df_second

In [3]:
loc = ['w','t']
label = ['downstairs','jogging','upstairs','walk_fast','walk_mod','walk_slow']

In [4]:
for loc_i in loc:
    for label_i in label:
        for num in range(26, 64):
            path = './%s/%s/0%g.csv'%(loc_i,label_i,num)
            new_path = './%s/%s/0%g_second.csv'%(loc_i,label_i,num)
            try:
                df = pd.read_csv(path)
                new_df = get_second_data(df)
                new_df.to_csv(new_path)
            except:
                continue

            

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

...

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

In [5]:
def combine_t_w(id, label,label_id):
    w_path = './w/%s/0%g_second.csv'%(label, id)
    t_path = './t/%s/0%g_second.csv'%(label, id)
    df_w = pd.read_csv(w_path)
    df_t = pd.read_csv(t_path)
    df_w = df_w.rename(columns={'x':'w_x','y':'w_y','z':'w_z'})
    df_t = df_t.rename(columns={'x':'t_x','y':'t_y','z':'t_z'})
    df_t_w = pd.merge(df_w,df_t)
    df_t_w['label_id'] = label_id
    return df_t_w

In [6]:
for label_i in label:
    for num in range(26, 64):
        path = './%s/%s/0%g.csv'%(loc_i,label_i,num)
        new_path = './%s/%s/0%g_second.csv'%(loc_i,label_i,num)
        try:
            df = pd.read_csv(path)
            new_df = get_second_data(df)
            new_df.to_csv(new_path)
        except:
            continue

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_second = df.groupby(['Minute','Second']).mean()
C:\Users\Xiaoyi Chen\AppData\Local\Temp\ipykernel_6932\3429072855.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which s

In [7]:
os.mkdir('./combined_data')
for label_i in label:
    os.mkdir('./combined_data/'+ label_i)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './combined_data'

In [9]:
column_names = ['Minute','Second','w_x','w_y','w_z','t_x','t_y','t_z']
df = pd.DataFrame(columns = column_names)

In [17]:
def sliding_window(window_length, stride, new_df):
    data = []
    label_id = new_df.iloc[0]['label_id']
    for i in range(0,len(new_df)-window_length,stride):
        start = i
        end = i + window_length
        np_data = np.array([])
        np_data = np.append(new_df.iloc[0:10][['w_x','w_y','w_z','t_x','t_y','t_z']].values.reshape(60,order='F'),label_id)
        data.append(np_data)
    data = np.array(data)
    return data

In [18]:
for num in range(26, 64):
    for i in range(len(label)):
        new_path = './combined_data/%s/0%g.csv'%(label[i],num)
        try:
            new_df = combine_t_w(num, label[i],i)
            window_length = 10
            stride = 1
            if num == 26 and i == 0:
                data_window = sliding_window(window_length, stride, new_df)
            else:
                temp_window = sliding_window(window_length, stride, new_df)
                data_window = np.append(data_window, temp_window,axis=0)
        except:
            continue

In [77]:
# Learn the embedding for each x,y,z
class linear_embedding(nn.Module):
    def __init__(self):
        super(linear_embedding, self).__init__()
        self.layer1 = nn.Linear(10,16)
        self.layer2 = nn.Linear(16,5)
        self.activation = nn.LeakyReLU(0.1)
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.activation(x)
        return x

In [85]:
class deep_model(nn.Module):
    def __init__(self):
        super(deep_model, self).__init__()
        self.le_w_x = linear_embedding()
        self.le_w_y = linear_embedding()
        self.le_w_z = linear_embedding()
        self.le_t_x = linear_embedding()
        self.le_t_y = linear_embedding()
        self.le_t_z = linear_embedding()
        self.layer1 = nn.Linear(30,16)
        self.layer2 = nn.Linear(16,6)
        self.activation = nn.LeakyReLU(0.1)
    def forward(self,x):
        w_x = self.le_w_x(x[:,0:10])
        w_y = self.le_w_y(x[:,10:20])
        w_z = self.le_w_z(x[:,20:30])
        t_x = self.le_t_x(x[:,30:40])
        t_y = self.le_t_y(x[:,40:50])
        t_z = self.le_t_z(x[:,50:60])
        embedding = torch.cat((w_x,w_y,w_z,t_x,t_y,t_z),dim = 1)
        x = self.layer1(embedding)
        x = self.activation(x)
        x = self.layer2(x)
        return x

In [96]:
np.random.shuffle(data_window)
X = torch.Tensor(data_window[:,0:60])
Y = torch.Tensor(data_window[:, 60].astype(int)).type(torch.long)
X_test = X[0:2000]
Y_test = Y[0:2000]
X_train = X[2000:]
Y_train = Y[2000:]

In [97]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, Y_train),batch_size = 16, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, Y_test),batch_size = 16, shuffle=True)

In [98]:
my_model = deep_model()
optimizer = torch.optim.Adam(my_model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=20000,gamma=0.5)
my_loss = nn.CrossEntropyLoss()

In [100]:
loss = 0
for epoch in range(10):
    my_model.train()
    for x,y in train_loader:
        optimizer.zero_grad()
        output = my_model(x)
        loss = my_loss(output,y)
        loss.backward()
        optimizer.step()
        print(loss)
    scheduler.step()
    my_model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            output = my_model(x)
            _,predicted = torch.max(output.data,1)
            correct += (predicted==y).sum().item()
        print('Accuracy is: %f'%(correct/2000*100))

tensor(0.0768, grad_fn=<NllLossBackward0>)
tensor(0.0103, grad_fn=<NllLossBackward0>)
tensor(0.0448, grad_fn=<NllLossBackward0>)
tensor(0.1142, grad_fn=<NllLossBackward0>)
tensor(0.1220, grad_fn=<NllLossBackward0>)
tensor(0.1772, grad_fn=<NllLossBackward0>)
tensor(0.1039, grad_fn=<NllLossBackward0>)
tensor(0.0690, grad_fn=<NllLossBackward0>)
tensor(0.0928, grad_fn=<NllLossBackward0>)
tensor(0.1606, grad_fn=<NllLossBackward0>)
tensor(0.0746, grad_fn=<NllLossBackward0>)
tensor(0.1197, grad_fn=<NllLossBackward0>)
tensor(0.1149, grad_fn=<NllLossBackward0>)
tensor(0.1056, grad_fn=<NllLossBackward0>)
tensor(0.0925, grad_fn=<NllLossBackward0>)
tensor(0.0736, grad_fn=<NllLossBackward0>)
tensor(0.1639, grad_fn=<NllLossBackward0>)
tensor(0.0914, grad_fn=<NllLossBackward0>)
tensor(0.0886, grad_fn=<NllLossBackward0>)
tensor(0.0799, grad_fn=<NllLossBackward0>)
tensor(0.0903, grad_fn=<NllLossBackward0>)
tensor(0.0478, grad_fn=<NllLossBackward0>)
tensor(0.0768, grad_fn=<NllLossBackward0>)
tensor(0.11

tensor(0.1173, grad_fn=<NllLossBackward0>)
tensor(0.0430, grad_fn=<NllLossBackward0>)
tensor(0.0382, grad_fn=<NllLossBackward0>)
tensor(0.0481, grad_fn=<NllLossBackward0>)
tensor(0.0699, grad_fn=<NllLossBackward0>)
tensor(0.1465, grad_fn=<NllLossBackward0>)
tensor(0.0434, grad_fn=<NllLossBackward0>)
tensor(0.1725, grad_fn=<NllLossBackward0>)
tensor(0.0773, grad_fn=<NllLossBackward0>)
tensor(0.0806, grad_fn=<NllLossBackward0>)
tensor(0.0592, grad_fn=<NllLossBackward0>)
tensor(0.0671, grad_fn=<NllLossBackward0>)
tensor(0.1324, grad_fn=<NllLossBackward0>)
tensor(0.0693, grad_fn=<NllLossBackward0>)
tensor(0.0204, grad_fn=<NllLossBackward0>)
tensor(0.0874, grad_fn=<NllLossBackward0>)
tensor(0.0846, grad_fn=<NllLossBackward0>)
tensor(0.0735, grad_fn=<NllLossBackward0>)
tensor(0.0430, grad_fn=<NllLossBackward0>)
tensor(0.1472, grad_fn=<NllLossBackward0>)
tensor(0.0976, grad_fn=<NllLossBackward0>)
tensor(0.0570, grad_fn=<NllLossBackward0>)
tensor(0.0739, grad_fn=<NllLossBackward0>)
tensor(0.31

tensor(0.1250, grad_fn=<NllLossBackward0>)
tensor(0.1164, grad_fn=<NllLossBackward0>)
tensor(0.0628, grad_fn=<NllLossBackward0>)
tensor(0.0714, grad_fn=<NllLossBackward0>)
tensor(0.0624, grad_fn=<NllLossBackward0>)
tensor(0.0590, grad_fn=<NllLossBackward0>)
tensor(0.0660, grad_fn=<NllLossBackward0>)
tensor(0.0650, grad_fn=<NllLossBackward0>)
tensor(0.0705, grad_fn=<NllLossBackward0>)
tensor(0.0753, grad_fn=<NllLossBackward0>)
tensor(0.0083, grad_fn=<NllLossBackward0>)
tensor(0.0554, grad_fn=<NllLossBackward0>)
tensor(0.0526, grad_fn=<NllLossBackward0>)
tensor(0.1098, grad_fn=<NllLossBackward0>)
tensor(0.1116, grad_fn=<NllLossBackward0>)
tensor(0.0429, grad_fn=<NllLossBackward0>)
tensor(0.0424, grad_fn=<NllLossBackward0>)
tensor(0.0763, grad_fn=<NllLossBackward0>)
tensor(0.0510, grad_fn=<NllLossBackward0>)
tensor(0.0604, grad_fn=<NllLossBackward0>)
tensor(0.0808, grad_fn=<NllLossBackward0>)
tensor(0.0923, grad_fn=<NllLossBackward0>)
tensor(0.0676, grad_fn=<NllLossBackward0>)
tensor(0.04

tensor(0.0547, grad_fn=<NllLossBackward0>)
tensor(0.0414, grad_fn=<NllLossBackward0>)
tensor(0.0361, grad_fn=<NllLossBackward0>)
tensor(0.1316, grad_fn=<NllLossBackward0>)
tensor(0.0397, grad_fn=<NllLossBackward0>)
tensor(0.0427, grad_fn=<NllLossBackward0>)
tensor(0.0506, grad_fn=<NllLossBackward0>)
tensor(0.0695, grad_fn=<NllLossBackward0>)
tensor(0.0295, grad_fn=<NllLossBackward0>)
tensor(0.0460, grad_fn=<NllLossBackward0>)
tensor(0.1443, grad_fn=<NllLossBackward0>)
tensor(0.1546, grad_fn=<NllLossBackward0>)
tensor(0.0908, grad_fn=<NllLossBackward0>)
tensor(0.0557, grad_fn=<NllLossBackward0>)
tensor(0.0834, grad_fn=<NllLossBackward0>)
tensor(0.0621, grad_fn=<NllLossBackward0>)
tensor(0.0656, grad_fn=<NllLossBackward0>)
tensor(0.0684, grad_fn=<NllLossBackward0>)
tensor(0.0221, grad_fn=<NllLossBackward0>)
tensor(0.1152, grad_fn=<NllLossBackward0>)
tensor(0.0490, grad_fn=<NllLossBackward0>)
tensor(0.0311, grad_fn=<NllLossBackward0>)
tensor(0.0162, grad_fn=<NllLossBackward0>)
tensor(0.08

tensor(0.0542, grad_fn=<NllLossBackward0>)
tensor(0.0368, grad_fn=<NllLossBackward0>)
tensor(0.0119, grad_fn=<NllLossBackward0>)
tensor(0.0733, grad_fn=<NllLossBackward0>)
tensor(0.0639, grad_fn=<NllLossBackward0>)
tensor(0.1173, grad_fn=<NllLossBackward0>)
tensor(0.0150, grad_fn=<NllLossBackward0>)
tensor(0.0974, grad_fn=<NllLossBackward0>)
tensor(0.0544, grad_fn=<NllLossBackward0>)
tensor(0.0197, grad_fn=<NllLossBackward0>)
tensor(0.1087, grad_fn=<NllLossBackward0>)
tensor(0.0565, grad_fn=<NllLossBackward0>)
tensor(0.0520, grad_fn=<NllLossBackward0>)
tensor(0.0711, grad_fn=<NllLossBackward0>)
tensor(0.0913, grad_fn=<NllLossBackward0>)
tensor(0.0592, grad_fn=<NllLossBackward0>)
tensor(0.0883, grad_fn=<NllLossBackward0>)
tensor(0.0758, grad_fn=<NllLossBackward0>)
tensor(0.0513, grad_fn=<NllLossBackward0>)
tensor(0.1498, grad_fn=<NllLossBackward0>)
tensor(0.0692, grad_fn=<NllLossBackward0>)
tensor(0.0225, grad_fn=<NllLossBackward0>)
tensor(0.0301, grad_fn=<NllLossBackward0>)
tensor(0.01

...

tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(8.4878e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(9.8444e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(4.6961e-05, grad_fn=<NllLossBackward0>)
tensor(7.8845e-05, grad_fn=<NllLossBackward0>)
tensor(8.7656e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(7.8562e-05, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(9.8767e-05, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<Nl

tensor(0.0004, grad_fn=<NllLossBackward0>)
tensor(0.0006, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(6.5504e-05, grad_fn=<NllLossBackward0>)
tensor(9.4159e-05, grad_fn=<NllLossBackward0>)
tensor(0.0004, grad_fn=<NllLossBackward0>)
tensor(0.0004, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(8.0863e-05, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)

tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(9.0073e-05, grad_fn=<NllLossBackward0>)
tensor(4.8668e-05, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(7.6790e-05, grad_fn=<NllLossBackward0>)
tensor(5.7230e-05, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(5.8711e-05, grad_fn=<NllLossBackward0>)
tensor(6.8461e-05, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(7.8767e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(6.1549e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(7.0983e-05, gra

tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(8.5906e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(8.0359e-05, grad_fn=<NllLossBackward0>)
tensor(5.5461e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(8.6860e-05, grad_fn=<NllLossBackward0>)
tensor(8.0364e-05, grad_fn=<NllLossBackward0>)
tensor(5.1299e-05, grad_fn=<NllLossBackward0>)
tensor(5.6543e-05, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(6.9831e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(6.5097e-05, grad_fn=<NllLossBackward0>)
tensor(4.3501e-05,

tensor(6.1236e-05, grad_fn=<NllLossBackward0>)
tensor(7.9630e-05, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(4.9434e-05, grad_fn=<NllLossBackward0>)
tensor(3.4562e-05, grad_fn=<NllLossBackward0>)
tensor(5.6293e-05, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0003, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(6.0108e-05, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0002, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(9.2927e-05, grad_fn=<NllLossBackward0>)
tensor(9.9007e-05, grad_fn=<NllLossBackward0>)
tensor(5.3643e-05, grad_fn=<NllLossBackward0>)
tensor(7.7161e-05, grad_fn=<NllLossBackward0>)
tensor(0.0001, grad_fn=<NllLossBackward0>)
tensor(0.0002,